Create matching

In [19]:
import pickle
from sklearn.utils import shuffle
import pandas as pd

with open( '/share/hel/datasets/fs/nlp/text_classification_bias/bios/BIOS.pkl', 'rb') as file:
    bios_dicts = pickle.load(file)
df = pd.DataFrame(bios_dicts)

df.drop(['path','name','start_pos','URI','title'], axis=1, inplace=True)
df['raw_title'] = df['raw_title'].map(lambda x: x.lower())
raw_bio =[]
bio = []
for i in range(len(df)):
    raw_bio.append(df['raw'][i].replace(df['raw_title'][i],'_'))
    bio.append(df['bio'][i].replace(df['raw_title'][i],'_'))
df['raw'] = raw_bio
df['bio'] = bio
print(len(df))


361634


In [20]:
import pandas as pd
import glob

path = '/share/hel/datasets/jobiqo/talent.com/UK/'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    frame = pd.read_csv(filename, index_col=None, header=0)
    li.append(frame)

uk_data = pd.concat(li, axis=0, ignore_index=True)
uk_data.drop(['company','city','state','country','date','time','jobid'], axis=1, inplace=True)
uk_data['title'] = uk_data['title'].map(lambda x: x.lower())
print(len(uk_data))
uk_data.drop_duplicates(subset="description",  keep=False, inplace=True)
new_uk_data = uk_data[uk_data['title'].isin(df.raw_title.unique())]
print(len(new_uk_data))

903011
2109


In [21]:
new_uk_data['description'] = new_uk_data['description'].replace('\n',' ', regex=True)

/tmp/deepak/ipykernel_59637/462123052.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_uk_data['description'] = new_uk_data['description'].replace('\n',' ', regex=True)


In [22]:
new_uk_data['title'].value_counts()

software engineer           522
senior software engineer    405
dentist                     279
accountant                  162
teacher                     152
architect                   146
nurse                       141
paralegal                    96
painter                      70
psychologist                 33
personal trainer             23
dietitian                    22
interior designer            18
photographer                 16
psychotherapist               4
journalist                    4
assistant professor           4
attorney                      3
associate professor           3
yoga instructor               2
model                         1
surgeon                       1
physician                     1
nutritionist                  1
Name: title, dtype: int64

In [23]:
new_uk_data = new_uk_data.drop(new_uk_data[new_uk_data['title']=='physician'].index)
new_uk_data = new_uk_data.drop(new_uk_data[new_uk_data['title']=='nutritionist'].index)
new_uk_data = new_uk_data.drop(new_uk_data[new_uk_data['title']=='surgeon'].index)
new_uk_data = new_uk_data.drop(new_uk_data[new_uk_data['title']=='model'].index)
new_uk_data = new_uk_data.drop(new_uk_data[new_uk_data['title']=='yoga instructor'].index)
new_uk_data = new_uk_data.drop(new_uk_data[new_uk_data['title']=='attorney'].index)
new_uk_data = new_uk_data.drop(new_uk_data[new_uk_data['title']=='associate professor'].index)
new_uk_data = new_uk_data.drop(new_uk_data[new_uk_data['title']=='journalist'].index)
new_uk_data = new_uk_data.drop(new_uk_data[new_uk_data['title']=='assistant professor'].index)
new_uk_data = new_uk_data.drop(new_uk_data[new_uk_data['title']=='psychotherapist'].index)

In [24]:
new_uk_data['title'].value_counts()

software engineer           522
senior software engineer    405
dentist                     279
accountant                  162
teacher                     152
architect                   146
nurse                       141
paralegal                    96
painter                      70
psychologist                 33
personal trainer             23
dietitian                    22
interior designer            18
photographer                 16
Name: title, dtype: int64

In [25]:
new_df = df[df['raw_title'].isin(uk_data.title.unique())]
print(len(new_df))

320038


In [26]:
import pickle

with open('/share/hel/datasets/jobiqo/talent.com/JobRec/bios.pkl', 'wb') as f:
    pickle.dump(new_df, f)

with open('/share/hel/datasets/jobiqo/talent.com/JobRec/uk_jobs.pkl', 'wb') as f:
    pickle.dump(new_uk_data, f)

BIOS counterfactual and cleaning

In [17]:
import pickle
import pandas as pd

with open( '/share/hel/datasets/jobiqo/talent.com/JobRec/bios.pkl', 'rb') as file:
    dicts = pickle.load(file)
bios = pd.DataFrame(dicts)
#bios = pd.DataFrame(dicts).reset_index()
#bios = bios.drop('index', axis=1)

import pickle
import pandas as pd

with open( '/share/hel/datasets/fs/nlp/text_classification_bias/bios/BIOS.pkl', 'rb') as file:
    dicts = pickle.load(file)
original_bios = pd.DataFrame(dicts)

import regex as re
from tqdm.autonotebook import tqdm

male_words = ['mr','his','he','him','himself']
female_words = ['mrs','hers','she','her','herself']

bio_list = []
counterfactual_list = []

for id in tqdm(bios.index):
    bio_string = bios['raw'][id]
    name_list = list(original_bios['name'][id])
    if len(name_list[1])==1:
        name_list[1] = name_list[1]+'.'
    if '' in name_list:
        name_list.remove('')


    bio_string = re.sub(' '.join(name_list), 'NAME',bio_string,flags=re.I)
    for name in name_list:
            bio_string = re.sub(r'\s'+name+r'\s', ' NAME ',bio_string,flags=re.I)
    if bios['gender'][id]=='M':
        bio_string = re.sub('NAME', 'Bob',bio_string)
        bio_list.append(bio_string)
        for i in range(len(male_words)):
            bio_string = re.sub(r'\s'+male_words[i]+r'\s', ' '+female_words[i]+' ',bio_string,flags=re.I)

        bio_string = re.sub('Bob', 'Alice',bio_string)
    elif bios['gender'][id]=='F':
        bio_string = re.sub('NAME', 'Alice',bio_string)
        bio_list.append(bio_string)
        for i in range(len(male_words)):
            bio_string = re.sub(r'\s'+female_words[i]+r'\s',' '+ male_words[i]+' ',bio_string,flags=re.I)

        bio_string = re.sub('Alice', 'Bob',bio_string)
    counterfactual_list.append(bio_string)

bios['bio'] = bio_list
bios['counter_bio'] = counterfactual_list


bios = bios.reset_index()
bios = bios.drop('index', axis=1)

i=0
for job in bios['raw_title'].unique():
    print(job)
    if i==0:
        new_bios = pd.concat([bios[(bios['gender']=='F') & (bios['raw_title']==job)].sample(n=50,random_state=1),bios[(bios['gender']=='M') & (bios['raw_title']==job)].sample(n=50,random_state=1)])
    else:
        new_bios = pd.concat([new_bios, pd.concat([bios[(bios['gender']=='F') & (bios['raw_title']==job)].sample(n=50,random_state=1),bios[(bios['gender']=='M') & (bios['raw_title']==job)].sample(n=50,random_state=1)])])
    i=i+1

new_bios = new_bios.reset_index()
new_bios = new_bios.drop('index', axis=1)

anonymous_bio = []
anonymous_counter_bio = []
for i in range(2400):
    anonymous_bio.append(re.sub(new_bios['raw_title'][i], '_', new_bios['bio'][i],flags=re.I))
    anonymous_counter_bio.append(re.sub(new_bios['raw_title'][i], '_', new_bios['counter_bio'][i],flags=re.I))

new_bios['bio'] = anonymous_bio
new_bios['counter_bio'] = anonymous_counter_bio

new_bios = new_bios.reset_index()
new_bios = new_bios.drop('index', axis=1)

with open('/share/hel/datasets/jobiqo/talent.com/JobRec/new_bios.pkl', 'wb') as f:
    pickle.dump(new_bios, f)

100%|██████████| 320038/320038 [03:29<00:00, 1528.76it/s]


assistant professor
journalist
architect
photographer
psychologist
teacher
nurse
associate professor
attorney
software engineer
painter
physician
personal trainer
surgeon
dietitian
dentist
model
psychotherapist
yoga instructor
interior designer
senior software engineer
accountant
nutritionist
paralegal


Split data

In [18]:
import pickle
import pandas as pd

with open( '/share/hel/datasets/jobiqo/talent.com/JobRec/new_bios.pkl', 'rb') as file:
    dicts = pickle.load(file)
bios = pd.DataFrame(dicts).reset_index()
bios.drop('index', axis=1, inplace=True)

In [19]:
bios['raw_title'].value_counts()

assistant professor         100
journalist                  100
nutritionist                100
accountant                  100
senior software engineer    100
interior designer           100
yoga instructor             100
psychotherapist             100
model                       100
dentist                     100
dietitian                   100
surgeon                     100
personal trainer            100
physician                   100
painter                     100
software engineer           100
attorney                    100
associate professor         100
nurse                       100
teacher                     100
psychologist                100
photographer                100
architect                   100
paralegal                   100
Name: raw_title, dtype: int64

In [20]:
#Load Data

from sklearn.model_selection import train_test_split

train, val_test = train_test_split(bios,random_state=100, test_size=0.30, stratify=bios['raw_title'], shuffle=True)
test, val = train_test_split(val_test,random_state=100, test_size=0.333, stratify=val_test['raw_title'], shuffle=True)
train = train.reset_index()
train.drop('index', axis=1, inplace=True)
val = val.reset_index()
val.drop('index', axis=1, inplace=True)
test = test.reset_index()
test.drop('index', axis=1, inplace=True)

In [21]:
len(test),len(val),len(train)

(480, 240, 1680)

In [22]:
import pickle

with open('/share/hel/datasets/jobiqo/talent.com/JobRec/train.pkl', 'wb') as f:
    pickle.dump(train, f)

with open('/share/hel/datasets/jobiqo/talent.com/JobRec/val.pkl', 'wb') as f:
    pickle.dump(val, f)

with open('/share/hel/datasets/jobiqo/talent.com/JobRec/test.pkl', 'wb') as f:
    pickle.dump(test, f)

Pickle to Jason File

In [26]:
import pickle
import pandas as pd

with open( '/share/hel/datasets/jobiqo/talent.com/JobRec/uk_jobs.pkl', 'rb') as file:
    dicts = pickle.load(file)
uk_jobs = pd.DataFrame(dicts)
uk_jobs = pd.DataFrame(dicts).reset_index()
uk_jobs = uk_jobs.drop('index', axis=1)

In [25]:
train

,raw,raw_title,gender,bio,counter_bio
0,Ricky Heldt is a Yoga Instructor for people ne...,yoga instructor,M,"Bob is a _ for people new to yoga, advanced or...","Alice is a _ for people new to yoga, advanced ..."
1,Grant Horner is a full-time Associate Professo...,associate professor,M,Bob is a full-time _ at The Masters College in...,Alice is a full-time _ at The Masters College ...
2,Marina Bondarevska is an Ukranian Model curren...,model,F,Alice is an Ukranian _ currently based in Thes...,Bob is an Ukranian _ currently based in Thessa...
3,Adam Quang is a registered _ with the Yoga All...,yoga instructor,M,Bob is a registered _ with the Yoga Alliance E...,Alice is a registered _ with the Yoga Alliance...
4,"Kevin Langford, PhD, is an _ of biology at Ste...",associate professor,M,"Bob, PhD, is an _ of biology at Stephen F. Aus...","Alice, PhD, is an _ of biology at Stephen F. A..."
...,...,...,...,...,...
1675,Joseph C Archer is a Nurse Practitioner Specia...,nurse,M,Joseph C Bob is a _ Practitioner Specialist in...,Joseph C Alice is a _ Practitioner Specialist ...
1676,Linda McCarthy is an _ in the office of the CT...,architect,F,Alice is an _ in the office of the CTO at Syma...,Bob is an _ in the office of the CTO at Symant...
1677,Steve Grunwell is a Senior Software Engineer a...,senior software engineer,M,Bob is a _ at Liquid Web. Specializing in Word...,Alice is a _ at Liquid Web. Specializing in Wo...
1678,Ms. Jill S. Mathews is a practicing _. Ms. Mat...,physician,F,Ms. Alice is a practicing _. Ms. Alice practic...,Ms. Bob is a practicing _. Ms. Bob practices m...


In [27]:
uk_jobs

,title,description
0,senior software engineer,The Senior .net Developer will be part of the ...
1,nurse,Registered Nurse - Good CQC - Paid breaks - ...
2,nurse,It’s more than a possibility Nurse - U...
3,dentist,We’re looking for an Associate Dentist to wor...
4,software engineer,Role Overview Looking for the next step...
...,...,...
2080,dentist,Description Associate Dentist required for ext...
2081,senior software engineer,MediaMathhelps the world's top brands deliver...
2082,photographer,"Photographer Altnagelvin, Londonderry BT..."
2083,personal trainer,Rare opportunity to join The Chapel - Luxury...


In [29]:
import pickle
import pandas as pd

with open( '/share/hel/datasets/jobiqo/talent.com/JobRec/uk_jobs.pkl', 'rb') as file:
    dicts = pickle.load(file)
uk_jobs = pd.DataFrame(dicts)
uk_jobs = pd.DataFrame(dicts).reset_index()
uk_jobs = uk_jobs.drop('index', axis=1)

home_path = '/share/hel/datasets/jobiqo/talent.com/JobRec/BM25/'

uk_jobs['id'] = ['doc'+str(i+1) for i in range(len(uk_jobs))]

#bios_true_gender = bios.drop(['raw','counter_bio'],axis='columns')
#bios_true_gender.rename(columns = {'bio':'contents'}, inplace = True)
#bios_true_gender.to_json(home_path + 'counterfactual_gender/bios.json', orient='records')


uk_jobs.rename(columns = {'description':'contents'}, inplace = True)
uk_jobs.to_json(home_path + 'uk_jobs.json', orient='records')

Get TSV

In [31]:
import pickle
import pandas as pd

with open('/share/hel/datasets/jobiqo/talent.com/JobRec/train.pkl', 'rb') as f:
    dicts = pickle.load(f)
    data = pd.DataFrame(dicts)
    data = data['bio']
    data.to_csv('/share/hel/datasets/jobiqo/talent.com/JobRec/train.tsv', header=False, sep="\t") 


with open('/share/hel/datasets/jobiqo/talent.com/JobRec/val.pkl', 'rb') as f:
    dicts = pickle.load(f)
    data = pd.DataFrame(dicts)
    data = data['bio']
    data.to_csv('/share/hel/datasets/jobiqo/talent.com/JobRec/val.tsv', header=False, sep="\t")

with open('/share/hel/datasets/jobiqo/talent.com/JobRec/test.pkl', 'rb') as f:
    dicts = pickle.load(f)
    data = pd.DataFrame(dicts)
    data = data['bio']
    data.to_csv('/share/hel/datasets/jobiqo/talent.com/JobRec/test.tsv', header=False, sep="\t")

GEt BM25 hits

In [32]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input /share/hel/datasets/jobiqo/talent.com/JobRec/BM25/ \
  --index /share/hel/datasets/jobiqo/talent.com/JobRec/BM25/indexes \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeDocvectors --storeRaw

2024-01-24 17:49:30,736 INFO  [main] index.IndexCollection (IndexCollection.java:391) - Setting log level to INFO
2024-01-24 17:49:30,738 INFO  [main] index.IndexCollection (IndexCollection.java:394) - Starting indexer...
2024-01-24 17:49:30,738 INFO  [main] index.IndexCollection (IndexCollection.java:395) - ============ Loading Parameters ============
2024-01-24 17:49:30,738 INFO  [main] index.IndexCollection (IndexCollection.java:396) - DocumentCollection path: /share/hel/datasets/jobiqo/talent.com/JobRec/BM25/
2024-01-24 17:49:30,738 INFO  [main] index.IndexCollection (IndexCollection.java:397) - CollectionClass: JsonCollection
2024-01-24 17:49:30,739 INFO  [main] index.IndexCollection (IndexCollection.java:398) - Generator: DefaultLuceneDocumentGenerator
2024-01-24 17:49:30,739 INFO  [main] index.IndexCollection (IndexCollection.java:399) - Threads: 1
2024-01-24 17:49:30,739 INFO  [main] index.IndexCollection (IndexCollection.java:400) - Language: en
2024-01-24 17:49:30,739 INFO  [

In [33]:
!python -m pyserini.search.lucene  --index /share/hel/datasets/jobiqo/talent.com/JobRec/BM25/indexes --topics /share/hel/datasets/jobiqo/talent.com/JobRec/train.tsv --output /share/hel/datasets/jobiqo/talent.com/JobRec/BM25/train_hits.txt  --bm25

Running /share/hel/datasets/jobiqo/talent.com/JobRec/train.tsv topics, saving to /share/hel/datasets/jobiqo/talent.com/JobRec/BM25/train_hits.txt...
100%|███████████████████████████████████████| 1680/1680 [02:17<00:00, 12.25it/s]


In [34]:
!python -m pyserini.search.lucene  --index /share/hel/datasets/jobiqo/talent.com/JobRec/BM25/indexes --topics /share/hel/datasets/jobiqo/talent.com/JobRec/val.tsv --output /share/hel/datasets/jobiqo/talent.com/JobRec/BM25/val_hits.txt  --bm25

Running /share/hel/datasets/jobiqo/talent.com/JobRec/val.tsv topics, saving to /share/hel/datasets/jobiqo/talent.com/JobRec/BM25/val_hits.txt...
100%|█████████████████████████████████████████| 240/240 [00:21<00:00, 11.18it/s]


In [36]:
!python -m pyserini.search.lucene  --index /share/hel/datasets/jobiqo/talent.com/JobRec/BM25/indexes --topics /share/hel/datasets/jobiqo/talent.com/JobRec/test.tsv --output /share/hel/datasets/jobiqo/talent.com/JobRec/BM25/test_hits.txt  --bm25

Running /share/hel/datasets/jobiqo/talent.com/JobRec/test.tsv topics, saving to /share/hel/datasets/jobiqo/talent.com/JobRec/BM25/test_hits.txt...
100%|█████████████████████████████████████████| 480/480 [00:43<00:00, 11.03it/s]
